<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/036_Market_Basket_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Analiza Koszykowa: Reguły Asocjacyjne (Apriori)

To technika używana przez każdy supermarket i sklep e-commerce (np. "Często kupowane razem" na Amazonie).

Musimy zrozumieć 3 metryki:

1.  **Support (Wsparcie):** Jak popularny jest zestaw?
    *   $Support(A) = \frac{\text{Transakcje z A}}{\text{Wszystkie transakcje}}$
    *   Niskie wsparcie = towar niszowy.
2.  **Confidence (Ufność):** Jeśli kupiłem A, to na ile % kupię B?
    *   $Confidence(A \to B) = \frac{\text{Transakcje z A i B}}{\text{Transakcje z A}}$
    *   To nasza "siła reguły".
3.  **Lift (Przyrost):** Czy ta zależność jest prawdziwa, czy to przypadek?
    *   $Lift = 1$: Brak związku (Niezależne).
    *   $Lift > 1$: A przyciąga B (Super!).
    *   $Lift < 1$: A odpycha B (Kto kupuje A, ten rzadziej kupuje B).

Algorytm **Apriori** przeszukuje bazę transakcji i wypluwa reguły typu: `{Chleb} -> {Masło}`.

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 1. DANE TRANSAKCYJNE (Lista list)
# Każdy wiersz to jeden paragon.
dataset = [
    ['Mleko', 'Cebula', 'Gałka muszkatołowa', 'Jajka', 'Jogurt'],
    ['Koper', 'Cebula', 'Gałka muszkatołowa', 'Jajka', 'Jogurt'],
    ['Mleko', 'Jabłko', 'Jajka'],
    ['Mleko', 'Jednorożec', 'Kukurydza', 'Jajka', 'Jogurt'],
    ['Kukurydza', 'Cebula', 'Cebula', 'Jajka', 'Lody', 'Jajka']
]

print("--- PRZYKŁADOWY PARAGON ---")
print(dataset[0])

--- PRZYKŁADOWY PARAGON ---
['Mleko', 'Cebula', 'Gałka muszkatołowa', 'Jajka', 'Jogurt']


## Krok 1: One-Hot Encoding (Format 0/1)

Algorytm Apriori nie rozumie napisów. Rozumie tabelę prawdy.
Musimy zamienić naszą listę zakupów na wielką tabelę, gdzie:
*   Kolumny = Wszystkie możliwe produkty w sklepie.
*   Wiersze = Paragony.
*   Wartość = True (kupił) / False (nie kupił).

Użyjemy do tego `TransactionEncoder`.

In [2]:
# Inicjalizacja Encodera
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

# Zamiana na DataFrame (żeby było ładnie widać)
df = pd.DataFrame(te_ary, columns=te.columns_)

print("--- TABELA PRAWDA/FAŁSZ (One-Hot) ---")
display(df)
print(f"Mamy {df.shape[0]} transakcji i {df.shape[1]} unikalnych produktów.")

--- TABELA PRAWDA/FAŁSZ (One-Hot) ---


,Cebula,Gałka muszkatołowa,Jabłko,Jajka,Jednorożec,Jogurt,Koper,Kukurydza,Lody,Mleko
0,True,True,False,True,False,True,False,False,False,True
1,True,True,False,True,False,True,True,False,False,False
2,False,False,True,True,False,False,False,False,False,True
3,False,False,False,True,True,True,False,True,False,True
4,True,False,False,True,False,False,False,True,True,False


Mamy 5 transakcji i 10 unikalnych produktów.


## Krok 2: Szukanie "Częstych Zbiorów" (Frequent Itemsets)

Teraz uruchamiamy **Apriori**.
Mówimy mu: *"Pokaż mi tylko te zestawy produktów, które występują w przynajmniej 60% paragonów (`min_support=0.6`)"*.

To odsieje rzadkie produkty (jak "Jednorożec" czy "Lody"), a zostawi bestsellery.

In [3]:
# use_colnames=True sprawia, że widzimy nazwy produktów, a nie numery kolumn
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

# Sortujemy od najczęstszych
frequent_itemsets = frequent_itemsets.sort_values(by="support", ascending=False)

print("--- NAJCZĘSTSZE ZESTAWY (Support > 60%) ---")
print(frequent_itemsets)

print("\nWniosek: Jajka są w 100% koszyków (Support=1.0). Królowie sprzedaży.")

--- NAJCZĘSTSZE ZESTAWY (Support > 60%) ---
   support         itemsets
1      1.0          (Jajka)
0      0.6         (Cebula)
2      0.6         (Jogurt)
3      0.6          (Mleko)
4      0.6  (Cebula, Jajka)
5      0.6  (Jogurt, Jajka)
6      0.6   (Mleko, Jajka)

Wniosek: Jajka są w 100% koszyków (Support=1.0). Królowie sprzedaży.


## Krok 3: Generowanie Reguł (Association Rules)

Mamy popularne zestawy (np. Jajka i Cebula).
Ale czy to znaczy, że **Jajka powodują kupno Cebuli**? Czy na odwrót?
A może po prostu wszyscy kupują jajka, więc siłą rzeczy lądują one z cebulą?

Tu wchodzi metryka **Lift**.
Generujemy reguły, które mają np. minimum 70% ufności (`min_threshold=0.7`).

In [4]:
# Tworzymy reguły na podstawie zbiorów z kroku 2
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Wybieramy tylko interesujące kolumny
cols = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules_clean = rules[cols].sort_values(by='lift', ascending=False)

print("--- WYKRYTE REGUŁY ASOCJACYJNE ---")
# antecedents = Jeśli kupiłeś TO...
# consequents = ...to kupisz TAMTO
display(rules_clean)

--- WYKRYTE REGUŁY ASOCJACYJNE ---


c:\Users\takze\OneDrive\Pulpit\python\venv\Lib\site-packages\mlxtend\frequent_patterns\association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


,antecedents,consequents,support,confidence,lift
0,(Cebula),(Jajka),0.6,1.0,1.0
1,(Jogurt),(Jajka),0.6,1.0,1.0
2,(Mleko),(Jajka),0.6,1.0,1.0


## 🧠 Podsumowanie: Dlaczego Lift jest najważniejszy?

Spójrz na wyniki.

1.  **Reguła:** `{Cebula} -> {Jajka}`
    *   **Confidence:** 1.0 (100%). Każdy kto kupił cebulę, kupił jajka.
    *   **Lift:** 1.0.
    *   **Interpretacja:** Lift = 1 oznacza NUDA. Jajka są tak popularne (są wszędzie), że kupno cebuli nie zwiększa szansy na kupno jajek. One i tak by tam były. To nie jest "zależność", to statystyka.

2.  **Gdybyś miał regułę:** `{Piwo} -> {Chipsy}`
    *   Gdyby **Lift** wynosił **2.5**, oznaczałoby to, że klient z piwem jest **2.5 raza bardziej skłonny** kupić chipsy niż losowy klient.
    *   **To jest Złoto.** To tutaj sklep stawia stojak z chipsami obok lodówki z piwem.

**Wniosek:**
W analizie koszykowej nie patrz ślepo na Confidence (Ufność). Patrz na **Lift**.
Lift > 1 to pieniądz.
Lift = 1 to przypadek.

In [5]:
display(rules_clean)

,antecedents,consequents,support,confidence,lift
0,(Cebula),(Jajka),0.6,1.0,1.0
1,(Jogurt),(Jajka),0.6,1.0,1.0
2,(Mleko),(Jajka),0.6,1.0,1.0
